This notebook is for visualizing each step of the CMMN process so that we can identify the intermediate representations of the whole thing before integrating into our setup.

For now, I will eschew the class based formulation that the original authors provided and just implement everything iteratively, as Dr. B suggested.

Also, this is now a branch of the ICWaves repo so that we can minimize any potential mistakes from separating things into a different package for now.

Here are the main steps for translating cue -> emotion:

1. Load in all source and target data. Emotion has 35 subj.s at 256 hz, cue has 12 subj.s at 500 hz. Load the resampled cue. Since this notebook is for visualization, for now do with a handful of subjects and then visualize with the full list.
2. Calculate the normed barycenter of the source data (emotion). Viz.
3. For each subject, calculate the filter to transform it to this barycenter. Viz.
4. Convolve, visualize before and after of cue.

Afterwards add subj. to subj. matching. Normally we can then run this through the clf and get a conf matrix, but Dr. B wants to see these first which I think is a good idea.

In [ ]:
# imports
import scipy
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.io import loadmat
from scipy.signal import butter, sosfilt, welch, freqz, sosfreqz

In [ ]:
# uncomment to viz whole thing
# emotion_subj_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35] # note subj. 22 is missing
# cue_subj_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# comment out to do more than a few. just a handful for testing the notebook
emotion_subj_list = ['1', '2', '3', '4']
frolich_subj_list = ['1', '2', '3', '4']

In [ ]:
# load in the data
emotion_data = []
for subj in emotion_subj_list:
    emotion_data.append(loadmat(f'../data/emotion_256/raw_emotion_data_and_IC_labels/subj-{subj}.mat')['data'])

frolich_data = []
for subj in frolich_subj_list:
    frolich_data.append(loadmat(f'../data/frolich_500/raw_frolich_data/subj-{subj}.mat')['X'])

In [ ]:
# viz functions
def psd(data, fs=256, nperseg=256):
    """
    Compute the Power Spectral Density (PSD) of the given data.

    Parameters:
    - data: list of numpy arrays, each containing EEG data for a subject
    - fs: sampling frequency (default: 256 Hz)
    - nperseg: length of each segment for Welch's method (default: 256)

    Returns:
    - f: array of sample frequencies (x-axis)
    - Pxx: power spectral density of the data (y-axis)
    """
    f, Pxx = welch(data, fs=fs, nperseg=nperseg)
    return f, Pxx

# Define a function to plot PSD
def plot_psd(data, fs=256, nperseg=256):
    """
    Plot the Power Spectral Density (PSD) of the given data.

    Note: right now this is straight averaging, not the barycenter norming I do later.
    If I want to do the norming, apply that first and then this will not average for me.

    Parameters:
    - data: list of numpy arrays, each containing EEG data for a subject
    - fs: sampling frequency (default: 256 Hz)
    - nperseg: length of each segment for Welch's method (default: 256)
    """
    plt.figure(figsize=(12, 8))

    for i, subj_data in enumerate(data):
        # If subj_data is multi-dimensional, average across the channels
        if subj_data.ndim > 1:
            subj_data = np.mean(subj_data, axis=0)

        f, Pxx = psd(subj_data, fs=fs, nperseg=nperseg)

        plt.plot(f, 10 * np.log10(Pxx), label=f'Subject {i+1}')

    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power Spectral Density (dB/Hz)')
    plt.title('PSD')
    plt.legend()
    plt.show()

In [ ]:
# barycenter comp functions
def compute_normed_barycenter(data):
    """
    Compute the normed barycenter of the given data.

    I normalize by each subj's sum, as recommended by Dr. B on slack.

    data: list of numpy arrays, each containing EEG data for a subject. Each array has shape (n_channels, n_samples).
        Note: these should all be 256 hz, which emotion is natively and cue has been resampled to.
    """

    normalized_psds = []
    for subj in data:
        f, Pxx = psd(subj)
        normalized_psds.append(Pxx / np.sum(Pxx))

    # now average all together
    barycenter = np.mean(normalized_psds, axis=0)

    return barycenter

Now that all data has been loaded and all functions are instantiated, the below code cells calculate and visualize each step.

In [ ]:
# plot the PSD of the emotion data (averaged over channels)

In [ ]:
# plot the normed barycenter of the emotion data
emotion_barycenter = compute_normed_barycenter(emotion_data)
plt.figure(figsize=(12, 8))
#

In [ ]:
# plot the PSD of the frolich data (averaged over channels)